# 预测用户在不同品类的产品购买力

## 一.建模背景

## 1.1 建模场景应用

在电商中，用户对于不同品类的各个产品有自己的偏好，直观来说就是对哪种产品的购买量最多。对于电商平台来说，了解用户在不同品类的产品购买力有助于为其客户创建个性化的产品，因此需要对用户在不同品类的各个产品的购买行为进行预测。

某零售公司想要了解顾客在不同类别的产品中的购买量，通过对上个月的产品对应的客户购买信息进行建模分析，来预测客户对各种产品的购买量，帮助公司为不同客户创建个性化的产品。

## 1.2 数据集说明

数据来源：https://datahack.analyticsvidhya.com/contest/black-friday/

该数据集包括客户人口统计信息，产品详细信息包括购买量总数。

特征|说明
-|-
客户信息
age|年龄
gender|性别
marital_status|婚姻状况
city_type|城市
stay_in_current_city|常居地年限
产品详细信息
product_id    |产品编号
product category|产品种类
purchase_amount |购买总数

## 1.3 建模环境

pyspark 回归算法

numpy

# 二.数据处理

## 2.1 导入数据

使用PySpark的读数据接口read.csv读取数据

In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession \
    .builder \
    .appName("test") \
    .config("spark.some.config.option", "setting") \
    .getOrCreate()

In [5]:
train = spark.read.csv('./train.csv', header=True, inferSchema=True)
test = spark.read.csv('./test.csv', header=True, inferSchema=True)

查看数据类型，使用**printSchema()**方法。在train上应用printSchema()，它将以树格式打印模式。

In [6]:
train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



预览数据集

In [7]:
train.head(5)

[Row(User_ID=1000001, Product_ID='P00069042', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID='P00248942', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6, Product_Category_3=14, Purchase=15200),
 Row(User_ID=1000001, Product_ID='P00087842', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=None, Product_Category_3=None, Purchase=1422),
 Row(User_ID=1000001, Product_ID='P00085442', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=14, Product_Category_3=None, Purchase=1057),
 Row(User_ID=1000002, Product_ID='P0

In [8]:
train.count()

550068

## 2.2 数据处理

### 2.2.1 插补缺失值

通过调用**drop()**，可以检查train上非空数值的个数，并进行测试。默认情况下，drop()方法将删除包含任何空值的行。还可以通过设置参数“all”,当且仅当该行所有参数都为null时以删除该行。

In [9]:
train.na.drop('any').count(),test.na.drop('any').count()

(166821, 71037)

使用-1来填充train和test的null值

In [10]:
train = train.fillna(-1)
test = test.fillna(-1)

### 2.2.2 分析数值特征

使用**describe()**方法查看Dataframe列的各种汇总统计信息，它显示了数字变量的统计信息；调用**show()**方法显示结果。

In [11]:
train.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            550068|            550068|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 6.41976991935542

调用**select()**并传递想要选择的列名。select()将显示所选列的结果。还可以通过提供用逗号分隔的列名，从数据框架中选择多个列。

In [12]:
train.select('User_ID', 'Age').show(5)

+-------+----+
|User_ID| Age|
+-------+----+
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000002| 55+|
+-------+----+
only showing top 5 rows



### 2.2.3 分析分类特征

为了建立一个模型，需要查看“train”和“test”分类特征的分布。调用**distinct()** 查看不同值。

In [13]:
train.select('Product_ID').distinct().count(), test.select('Product_ID').distinct().count()#train,test中Product_ID不同类别的数量

(3631, 3491)

In [14]:
diff_cat_in_train_test = test.select('Product_ID').subtract(train.select('Product_ID'))#test和train中Product_ID不同的类别

diff_cat_in_train_test.distinct().count()#在test中而不在train中的类别

46

In [15]:
diff_cat_in_train_test.distinct().show(5)

+----------+
|Product_ID|
+----------+
| P00322642|
| P00300142|
| P00077642|
| P00249942|
| P00294942|
+----------+
only showing top 5 rows



### 2.2.4 将分类特征转换为标签

**StringIndexer()** 将分类列转换为标签

In [16]:
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_id_trans' )#该转换将标签的Product_ID列 编码为 标签索引的列
labeller = plan_indexer.fit(train)#fit()应用于train中，构建标签

运用labeller，调用**transform()** 转换train和test的Dataframe。将转换结果存储在Train1和Test1中

In [17]:
Train1 = labeller.transform(train)
Test1 = labeller.transform(test)
Train1.show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|product_id_trans|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|           765.0|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|           183.0|
+-------+----------+------+----+----------+-------------+--------------------------+-

In [18]:
Train1.select('product_id_trans').show(2)

+----------------+
|product_id_trans|
+----------------+
|           765.0|
|           183.0|
+----------------+
only showing top 2 rows



# 三. 特征生成

In [19]:
from pyspark.ml.feature import RFormula
formula = RFormula(formula="Purchase ~ Age+ Occupation +City_Category+Stay_In_Current_City_Years+Product_Category_1+Product_Category_2+ Gender",
                  featuresCol="features",labelCol="label")
#指定依赖列和独立列
#必须为features列和label列指定名称

In [20]:
t1 = formula.fit(Train1)#应用
train1 = t1.transform(Train1)#转换
test1 = t1.transform(Test1)
train1.show(2)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+--------------------+-------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|product_id_trans|            features|  label|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+----------------+--------------------+-------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|           765.0|(16,[6,10,13,14],...| 8370.0|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|               

可以看到train1和test1有两个额外的列，称为features和label，并对在公式formula中指定的列进行标记(featuresCol= features和labelCol= label)。
train1和test1中的features列中的所有分类变量都被转换为数值，可以查看train1和test1中的列特性和标签。

In [21]:
train1.select('features').show(2)

+--------------------+
|            features|
+--------------------+
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
+--------------------+
only showing top 2 rows



In [22]:
train1.select('label').show(2)

+-------+
|  label|
+-------+
| 8370.0|
|15200.0|
+-------+
only showing top 2 rows



# 四. 进行建模

在应用RFormula和转换Dataframe之后，根据这些数据开发机器学习模型。
以下应用随机森林回归，导入一个在pyspark.ml中定义的随机森林回归器。然后建立一个叫做rf的模型。将使用随机森林算法的默认参数。

In [23]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()

将train1数据划分为train_cv和test_cv进行交叉验证

In [24]:
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])

model1 = rf.fit(train_cv)  #在train_cv上建立模型
predictions = model1.transform(test_cv) #在test_cv上进行预测。结果将保存在predictions中

# 五. 评估模型

评估对test_cv的预测，看看rmse和mse是多少

In [25]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator() #创建对象 对模型求值
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse"})# 指定度量标准
import numpy as np
np.sqrt(mse),mse #rmse mse

(3781.6711152346434, 14301036.42380003)

In [26]:
model = rf.fit(train1) # 在所有的train1数据集上再次训练这个模型
predictions1 = model.transform(test1)

In [27]:
df = predictions1.selectExpr("User_ID as User_ID", "Product_ID as Product_ID", 'prediction as Purchase') # 测试集预测结果 保存
df.toPandas().to_csv('./submission.csv')